In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

tf.set_random_seed(0)

def unpickle(file):
    import cPickle
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

In [2]:
data_batch_1 = unpickle("/cifar-10/data_batch_1")
data_batch_2 = unpickle("/cifar-10/data_batch_2")
data_batch_3 = unpickle("/cifar-10/data_batch_3")
data_batch_4 = unpickle("/cifar-10/data_batch_4")
test_batch = unpickle("/cifar-10/test_batch")
train_x = np.concatenate((data_batch_1['data'],data_batch_2['data'],data_batch_3['data'],data_batch_4['data']))
train_y = np.concatenate((data_batch_1['labels'],data_batch_2['labels'],data_batch_3['labels'],data_batch_4['labels']))

In [3]:
# 1. Define Variables and Placeholders
X = tf.placeholder(tf.float32, [None, 32*32*3])
Y_ = tf.placeholder(tf.float32, [None, 10])
x_image = tf.reshape(X, [-1,32,32,3])
pkeep = tf.placeholder(tf.float32)

# Convolutional layer 1
W1 = tf.Variable(tf.truncated_normal([5, 5, 3, 4], stddev=0.1))
B1 = tf.Variable(tf.constant(0.1, shape=[4]))
Y1 = tf.nn.relu(tf.nn.conv2d(x_image, W1, strides=[1, 1, 1, 1], padding='SAME') + B1)
Y1_drop = tf.nn.dropout(Y1, pkeep)

# Convolutional layer 2
W2 = tf.Variable(tf.truncated_normal([5, 5, 4, 8], stddev=0.1))
B2 = tf.Variable(tf.constant(0.1, shape=[8]))
Y2 = tf.nn.relu(tf.nn.conv2d(Y1_drop, W2, strides=[1, 2, 2, 1], padding='SAME') + B2)
Y2_drop = tf.nn.dropout(Y2, pkeep)
                 
# Convolutional layer 3
W3 = tf.Variable(tf.truncated_normal([4, 4, 8, 12], stddev=0.1))
B3 = tf.Variable(tf.constant(0.1, shape=[12]))
Y3 = tf.nn.relu(tf.nn.conv2d(Y2_drop, W3, strides=[1, 2, 2, 1], padding='SAME') + B3)
Y3_drop = tf.nn.dropout(Y3, pkeep)

# Dense layer
W = tf.Variable(tf.truncated_normal([8*8*12, 200], stddev=0.1))
B = tf.Variable(tf.constant(0.1, shape=[200]))
Y3_flat = tf.reshape(Y3_drop, [-1, 8*8*12])
Y_fc1 = tf.nn.relu(tf.matmul(Y3_flat, W) + B)
Y_drop = tf.nn.dropout(Y_fc1, pkeep)

# Readout layer
W_fc2 = tf.Variable(tf.truncated_normal([200, 10], stddev=0.1))
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]))
Y_conv = tf.matmul(Y_fc1, W_fc2) + b_fc2
Y = tf.nn.softmax(Y_conv)

In [4]:
# 3. Define the loss function
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(Y_conv, Y_) # calculate cross-entropy with logits
cross_entropy = tf.reduce_mean(cross_entropy)

In [5]:
# 4. Define the accuracy
correct_prediction = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [6]:
# 5. Train with an Optimizer
DECAY_STEP = 1000
GLOBAL_STEP = tf.Variable(0, trainable=False)
DECAY_RATE = 0.75
LR = 0.001
learning_rate = tf.train.exponential_decay(LR, GLOBAL_STEP, DECAY_STEP, DECAY_RATE, staircase=True)
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy, global_step=GLOBAL_STEP)

In [7]:
# initialize
EPOCHS = 10000
ROUNDS = 100
BATCH_SIZE = 100
DROPOUT = 0.75
accuracies = []
losses = []
train_accuracies = []
train_losses = []
fig, (ax1, ax2) = plt.subplots(1, 2)
plt.subplots_adjust(wspace=.3)
fig.set_size_inches(10, 4)
ax1.set_ylabel("Loss")
ax1.set_xlabel("Epochs")
ax2.set_ylabel("Accuracy")
ax2.set_xlabel("Epochs")

init = tf.initialize_all_variables() 
sess = tf.Session()
sess.run(init)

In [8]:
def next_batch(train_x,train_y,batch_size):
    for i in range(0, train_x.shape[0], batch_size):
        yield train_x[i:(i+batch_size),:],train_y[i:(i+batch_size),:]

In [9]:
# One-hot encoding
num_labels = 10
indexedInputs = [[i, train_y[i]] for i in range(len(train_y))]
train_y_encoded = sess.run(tf.sparse_to_dense(indexedInputs, [len(train_y),num_labels], 1))
indexedInputs = [[i, test_batch['labels'][i]] for i in range(len(test_batch['labels']))]
test_y_encoded = sess.run(tf.sparse_to_dense(indexedInputs, [len(test_batch['labels']),num_labels], 1))

In [ ]:
batch = next_batch(train_x,train_y_encoded,BATCH_SIZE)
for i in range(EPOCHS):
  try:
    batch_x, batch_y = batch.next()
  except StopIteration:
    batch = next_batch(train_x,train_y_encoded,BATCH_SIZE)
    batch_x, batch_y = batch.next()
  sess.run(train_step,feed_dict={X: batch_x, Y_: batch_y, pkeep:DROPOUT})
  if (i % ROUNDS == 0):
    train_accuracy, train_loss = sess.run([accuracy, cross_entropy], feed_dict={X:batch_x, Y_: batch_y, pkeep:DROPOUT})
    print("step %d, training accuracy %g"%(i, train_accuracy))
    test_accuracy = sess.run(accuracy, feed_dict={X: test_batch['data'], Y_: test_y_encoded, pkeep:1.0})
    print("test accuracy %g"%test_accuracy)
    test_loss = sess.run(cross_entropy, feed_dict={X: test_batch['data'], Y_: test_y_encoded,pkeep:1.0})
    print("test loss %g"%test_loss)
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    losses.append(test_loss)
    accuracies.append(test_accuracy)

step 0, training accuracy 0.05
test accuracy 0.1089
test loss 21.4695
step 100, training accuracy 0.16
test accuracy 0.1202
test loss 2.32848
step 200, training accuracy 0.11
test accuracy 0.1328
test loss 2.29852
step 300, training accuracy 0.18
test accuracy 0.1016
test loss 2.30146
step 400, training accuracy 0.13
test accuracy 0.0996
test loss 2.30255
step 500, training accuracy 0.16
test accuracy 0.1025
test loss 2.3022
step 600, training accuracy 0.12
test accuracy 0.0943
test loss 2.3027
step 700, training accuracy 0.13
test accuracy 0.1153
test loss 2.29541
step 800, training accuracy 0.11
test accuracy 0.1213
test loss 2.2937
step 900, training accuracy 0.2
test accuracy 0.1532
test loss 2.27423
step 1000, training accuracy 0.11
test accuracy 0.1438
test loss 2.26463
step 1100, training accuracy 0.19
test accuracy 0.1576
test loss 2.2371
step 1200, training accuracy 0.16
test accuracy 0.1687
test loss 2.23694
step 1300, training accuracy 0.13
test accuracy 0.1685
test loss 2.2